# Access AWS CESM2 using the AWS open data origin data 

In [13]:
# Display output of plots directly in Notebook
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

import intake
import numpy as np
import pandas as pd
import xarray as xr
import re
# import nest_asyncio
# nest_asyncio.apply()
import aiohttp

In [14]:
import fsspec.implementations.http as fshttp
from pelicanfs.core import PelicanFileSystem, PelicanMap, OSDFFileSystem 

In [15]:
import dask 
from dask_jobqueue import SLURMCluster
from dask.distributed import Client
from dask.distributed import performance_report

In [16]:
#
rda_url    =  'https://data.rda.ucar.edu/'
intake_url = rda_url + 'harshah/intake_catalogs/cesm2-lens-osdf/aws-cesm2-le.json'

In [17]:
# # Spin up SLURM cluster
# job_extra = ['--qos=cf_lowprio','--account=ac_cumulus'] 
# cluster =  SLURMCluster(queue="cf1", cores=5, walltime='6:00:00', 
#                 local_directory='/global/scratch/users/harsha/dask_space/', 
#                 log_directory='/global/scratch/users/harsha/dask_space/', 
#                 job_extra_directives=job_extra, interface='eth0', memory="192GB") 
# client  = Client(cluster) 
# cluster.scale_up(4)

### Access the data from the AWS bucket using intake

In [18]:
osdf_catalog = intake.open_esm_datastore(intake_url)
osdf_catalog

,unique
Unnamed: 0,322
variable,53
long_name,51
component,4
experiment,2
forcing_variant,2
frequency,3
vertical_levels,3
spatial_domain,3
units,20


In [19]:
osdf_catalog.df['path'].head().values

array(['osdf:///aws-opendata/us-west-2/ncar-cesm2-lens/atm/daily/cesm2LE-historical-cmip6-FLNS.zarr',
       'osdf:///aws-opendata/us-west-2/ncar-cesm2-lens/atm/daily/cesm2LE-historical-cmip6-FLNSC.zarr',
       'osdf:///aws-opendata/us-west-2/ncar-cesm2-lens/atm/daily/cesm2LE-historical-cmip6-FLUT.zarr',
       'osdf:///aws-opendata/us-west-2/ncar-cesm2-lens/atm/daily/cesm2LE-historical-cmip6-FSNS.zarr',
       'osdf:///aws-opendata/us-west-2/ncar-cesm2-lens/atm/daily/cesm2LE-historical-cmip6-FSNSC.zarr'],
      dtype=object)

In [20]:
osdf_catalog_temp = osdf_catalog.search(variable ='TREFHTMX', frequency ='daily')
osdf_catalog_temp

,unique
Unnamed: 0,4
variable,1
long_name,1
component,1
experiment,2
forcing_variant,2
frequency,1
vertical_levels,1
spatial_domain,1
units,1


In [ ]:
%%time
#dsets = osdf_catalog_temp.to_dataset_dict(storage_options={'anon':True})
dsets = osdf_catalog_temp.to_dataset_dict()


--> The keys in the returned dictionary of datasets are constructed as follows:
	'component.experiment.frequency.forcing_variant'


In [ ]:
historical_smbb = dsets['atm.historical.daily.smbb']
historical_smbb

In [ ]:
%%time
historical_smbb.TREFHTMX.isel(member_id=0,time=0).plot()